In [1]:
import glob
import gzip
import tqdm.notebook as tq
import json
import numpy as np

# reading and combining files
ZIPFILES='*.csv'
filelist = glob.glob(ZIPFILES)
import pandas as pd
df = pd.DataFrame()
for gzfile in tq.tqdm(filelist):
    if len(df)==0:
        df=pd.read_csv(gzfile)
    else:
        data = pd.read_csv(gzfile)
        df=pd.concat([data,df])
df=df.reset_index().drop(["index"], axis=1)
df = df[df['fips'].notna()]
df['cases'] = df['cases'].fillna(0)
df['deaths']=df['deaths'].fillna(0)

  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
df["fips"]=[str(int(i)) for i in df.fips]
df["fips"]=(pd.Series(df.fips)).str.pad(width=5,fillchar='0')
df['date']=pd.to_datetime(df['date'], format='%Y-%m-%d')
df=df.sort_values(by='date')
df["cases"]=[int(i) for i in df["cases"]]
df["deaths"]=[int(i) for i in df["deaths"]]

In [3]:
df = df[df['deaths'].notna()]
df = df[df['cases'].notna()]
list_of_fips=list(np.unique(np.array(df.fips)))
df_new=pd.DataFrame()
for county in tq.tqdm(list_of_fips):
    df_temp=df[df['fips']==str(county)]
    df_temp=df_temp.sort_values(by='date')
    df_temp=df_temp.reset_index().drop(["index"], axis=1)
    temp=[(int((df_temp['cases'])[i]) - int((df_temp['cases'])[i-1])) for i in range(1,len(df_temp))]
    temp.append(int(df_temp.cases[0]))
    df_temp['cases']=temp
    temp=[(int((df_temp['deaths'])[i]) - int((df_temp['deaths'])[i-1])) for i in range(1,len(df_temp))]
    temp.append(int(df_temp.deaths[0]))
    df_temp['deaths']=temp
    if len(df_new)==0:
        df_new=df_temp
    else:
        df_new=pd.concat([df_new,df_temp])

df_new=df_new.reset_index().drop(["index"], axis=1)
df_new['deaths'] = df_new['deaths'].fillna(0)
df_new['cases'] = df_new['cases'].fillna(0)
df_new['date']=(df_new['date']).dt.to_period('M')


  0%|          | 0/3220 [00:00<?, ?it/s]

In [5]:
aggregated=(df_new.groupby(['date','county','state','fips'],as_index=False).agg(monthly_cases=('cases', 'sum'),monthly_deaths=('deaths','sum')))
aggregated=aggregated.reset_index().drop(["index"], axis=1)
aggregated['monthly_cases'] = aggregated['monthly_cases'].mask(aggregated['monthly_cases'] < 0, 0)
aggregated['monthly_deaths'] = aggregated['monthly_deaths'].mask(aggregated['monthly_deaths'] < 0, 0)

In [6]:
aggregated.to_csv("covid_data_clean.csv",index=False)


In [7]:
aggregated

,date,county,state,fips,monthly_cases,monthly_deaths
0,2020-01,Cook,Illinois,17031,1,0
1,2020-01,Los Angeles,California,06037,0,0
2,2020-01,Maricopa,Arizona,04013,0,0
3,2020-01,Orange,California,06059,0,0
4,2020-01,Santa Clara,California,06085,0,0
...,...,...,...,...,...,...
97979,2022-09,Yuma,Arizona,04027,100,1
97980,2022-09,Yuma,Colorado,08125,13,0
97981,2022-09,Zapata,Texas,48505,52,0
97982,2022-09,Zavala,Texas,48507,26,1


In [8]:
aggregated.sort_values(by="monthly_cases",ascending=False)

,date,county,state,fips,monthly_cases,monthly_deaths
70766,2022-01,Los Angeles,California,06037,971512,1361
28932,2020-12,Los Angeles,California,06037,382380,2852
32150,2021-01,Los Angeles,California,06037,330454,6302
70840,2022-01,Maricopa,Arizona,04013,311301,1082
70253,2022-01,Harris,Texas,48201,305249,460
...,...,...,...,...,...,...
42897,2021-04,Texas,Missouri,29215,0,1
42892,2021-04,Terrell,Texas,48443,0,0
42856,2021-04,Sutton,Texas,48435,0,0
42832,2021-04,Sullivan,Missouri,29211,0,0
